# FASTAPI Tutorial

Fastapi is a python frame work that actually increases the phase of api development, thanks for the asynchronous execution of request using the UVICORN etc.

some of the advantages over other python frameworks are

- **ASGI**, or the Asynchronous Server Gateway Interface, is a Python standard that defines how web servers communicate with asynchronous Python web frameworks and applications. As the successor to the WSGI (Web Server Gateway Interface), ASGI enables non-blocking, real-time communication features like WebSockets and server-sent events, which were not supported by the synchronous WSGI standard. It provides a standardized interface for asynchronous servers and applications, allowing for high-throughput, real-time web applications with better performance and scalability in Python.

- **Typing Hinting** using pydantic library. This increases the productivity of the programmers while coding. Typing Hinting is nothing but showing recommentions on methods and properties based on the data-type

- **Auto Docs**  using swagger UI and redoc. This helps to document the api easily and gives a greate UI for debugging as well!

###  About UVICORN
Steps inside Uvicorn:
1. Create config → builds settings (host, port, app reference, logging).
2. Initialize Server → sets up sockets, protocols, HTTP parsing (via h11 or httptools), and lifespan handlers.
3. Bind to host/port → since you passed 0.0.0.0:8000, it opens a listening socket for connections on all interfaces.
4. Start asyncio tasks → schedules the ASGI app (app) inside the loop, handling HTTP requests asynchronously.
5. Event loop keeps running → but because of nest_asyncio, the notebook doesn’t crash — it keeps processing notebook commands too.

### About Pydantic
All the type casting, data validation, type hinting is all done by pydantic library

### How to run 

There are severals ways to run the fastAPI app.
1. Using UVICORN
    - uvicorn \<filename\>:\<fastapi Instance\> --reload -> on cmd
    - uvicorn.run(\<fastapi Instance\>, host="0.0.0.0", port=8000) -> on code
2. Using FastAPI CMD
    - fastapi dev \<filename\> -> on cmd to run on development mode
    - fastapi run \<filename\> -> on cmd to run on production mode

## Basics

In [30]:
from fastapi import FastAPI

In [31]:
app = FastAPI() # Here the app is the instance of the FASTAPI

In [32]:
@app.get("/") #  Here @ is a decorator used to  decorate the function below 
def base_route():
    return "Welcome to the base page!"

# Here the get is the operation, '/' is the path or route or endpoint, and the function is called path operation function

#### Dynamic Route or Path Parameters
**Note**: The dynamic route must always present after the static routes. This ensures that the dynamic route won't recognize the static route and process the unexpected data. FastAPI always matches the path from top to bottom so placing the dynamic route after the static route will result in the expected outcome

In [33]:
@app.get("/blog/{id}")
def get_blog(id: int): # If the id is not integer fastapi will automatically throw an error a as response
    return {"data": f"data of blog {id}"}

#### Query and optional Parameter

In [34]:
@app.get("/blog")
def get_blog_with_limit(limit: int): # Here limit is a query parameter which is specified in the URL using '?' and '&'
    return {"Query Param":limit}

@app.get("/blog/optional")
def get_blog_with_optional_param(param: str | None=None):  # Here '|' is used to mention that it is optional 
    return {"Optional Param": param}


#### Request Body

In [35]:
from pydantic import BaseModel

class RequestBody(BaseModel):
    param1: int
    param2: str
    param3: bool

@app.post("/blog/create")
def creat_blog(request: RequestBody):
    return {"Data": request}


#### Running FastAPI inside Jupyter Notebook

To run fastapi inside jupyter notebook we need to use nest_asyncio.
- uvicorn.run() internally calls asyncio.run().
- But Jupyter/IPython already runs an event loop for you.
- Python forbids nesting asyncio.run() inside an already running loop, so you get:
    - **RuntimeError: asyncio.run() cannot be called from a running event loop**

1. Jupyter already has an event loop
- Jupyter (IPython kernel) runs an asyncio event loop in the background to manage code execution, message passing, widgets, etc.
Normally, calling asyncio.run() inside an already running loop raises:
RuntimeError: asyncio.run() cannot be called from a running event loop

2. nest_asyncio.apply()
-  This function patches the current event loop so that it can be nested.
Normally, asyncio forbids re-entering the same loop (asyncio.run always expects a fresh loop).
After patching, the existing Jupyter event loop can re-enter itself — allowing libraries like Uvicorn, aiohttp, etc. to run without crashing.
Think of it as: “let me allow recursion on the event loop.”

In [36]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [7084]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54310 - "GET /blog/optional?param=12 HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:54310 - "GET /blog/docs HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:54310 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54310 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:61296 - "POST /blog/create HTTP/1.1" 200 OK
INFO:     127.0.0.1:61296 - "POST /blog/create HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7084]
